# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 05: Batch Predictions</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/bitcoin/5_batch_predictions.ipynb)

## 🗒️ This notebook is divided into the following sections: 

1. Load the deployment
2. Retrieve batch data
3. Batch prediction using deployment

![part3](../../images/03_model.png) 

### <span style="color:#ff5f27;"> 📝 Imports</span>

In [ ]:
import pandas as pd

import tensorflow as tf 
import inspect 

---

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;"> ✨ Load Batch Data of last days</span>

First, you will need to fetch the training dataset that you created in the previous notebook.

In [ ]:
feature_view = fs.get_feature_view("bitcoin_feature_view", 1)
feature_view.init_batch_scoring(training_dataset_version=1)
feature_df = feature_view.get_batch_data().drop('unix',axis=1)
feature_df.head()

---

## <span style="color:#ff5f27;">🚀 Use the model to predict electricity prices</span>

In [ ]:
mr = project.get_model_registry()

model = mr.get_model("bitcoin_price_model", version = 1)
model_dir = model.download()

loaded_model = tf.saved_model.load(model_dir)
serving_function = loaded_model.signatures["serving_default"]

In [ ]:
predictions = serving_function(tf.constant(feature_df.values.reshape(-1, feature_df.shape[0], feature_df.shape[1]), tf.float32))['dense_1'].numpy()
predictions[:5]

Recall that you applied transformation functions, such as min max scaler and laber encoder. 

Now you want to transform them back to human readable format.

In [ ]:
feature_view.init_serving(1)
td_transformation_functions = feature_view._single_vector_server._transformation_functions

pred_y = pd.DataFrame(predictions, columns=["close"])

for feature_name in td_transformation_functions:
    if feature_name == "close":
        td_transformation_function = td_transformation_functions[feature_name]
        sig, foobar_locals = inspect.signature(td_transformation_function.transformation_fn), locals()
        param_dict = dict([(param.name, param.default) for param in sig.parameters.values() if param.default != inspect._empty])
        if td_transformation_function.name == "min_max_scaler":
            pred_y[feature_name] = pred_y[feature_name].map(lambda x: x*(param_dict["max_value"]-param_dict["min_value"])+param_dict["min_value"])
            
pred_y.head()        

---